In [18]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import re
import time;

In [38]:
#Chrome Driver path
path_driver = "chromedriver"

driver = webdriver.Chrome(path_driver)

In [27]:
#Classes for user and bio
class User:
    def __init__(self, user_id, name, 
                 followers, following, total_posts, bio,
                 bio_emails,profile_picture, privacy,
                influencer, buisness):
        self.user_id = user_id
        self.name = name
        self.followers = followers
        self.following = following
        self.total_posts = total_posts
        self.bio = bio
        self.profile_picture = profile_picture
        self.privacy_type = privacy
        self.influencer = influencer
        self.buisness = buisness
        self.bio_emails = bio_emails

class Bio:
    def __init__(self, bio_description, bio_ids, bio_links, bio_tags):
        self.bio_description = bio_description
        self.bio_ids = bio_ids
        self.bio_links = bio_links
        self.bio_tags = bio_tags

In [37]:
#initial structure
base_url = "https://www.instagram.com/"

user_id = []
name = []
followers = []
following = []
total_posts = []
profile_picture = []
privacy_type = []
bio_description = []
bio_links = []
bio_ids = []
bio_tags = []
influencer = []
buisness = []
bio_emails = []

In [39]:
#insert file {}.txt
input_file = open("accounts.txt", "r")

webScrape(input_file)

Local Start time : time.struct_time(tm_year=2020, tm_mon=9, tm_mday=16, tm_hour=18, tm_min=44, tm_sec=24, tm_wday=2, tm_yday=260, tm_isdst=0)
MEN'S FASHION & STYLE
24675
False
dynamic.malay
not valid
kratijain1606
not valid
MEN FASHION / FITNESS
164234
False
avni
not valid
mishikaa9
not valid
Indori Zayka®
229137
False
KATY PERRY
107164331
False
Miley Cyrus
115725428
False
Nike
121879716
False
Barbie
122961900
False
Jennifer Lopez
131545784
False
Taylor Swift
140666154
False
ene10ta Érre 🇧🇷 👻 neymarjr
142204958
False
National Geographic
145218705
False
Beyoncé
154875770
False
Leo Messi
166921078
False
Selena Gomez
193419872
False
therock
198401588
False
Cristiano Ronaldo
237994187
False
Local Finish time : time.struct_time(tm_year=2020, tm_mon=9, tm_mday=16, tm_hour=18, tm_min=45, tm_sec=7, tm_wday=2, tm_yday=260, tm_isdst=0)


In [15]:
def webScrape(input_file):
    localtime = time.localtime(time.time())
    print ("Local Start time :", localtime)

    for line in input_file:
        user = getAccountDataAsync(base_url + line)
    
        if user != None:
            print(user.name)
            print(user.followers)
            print(user.privacy_type)
        
            addUser(user)
        else:
            print(line+"not valid")

    generateExcelFromList()        

    localtime = time.localtime(time.time())
    print ("Local Finish time :", localtime)

In [32]:
def generateExcelFromList():
    df = pd.DataFrame({'userid':user_id,'name':name,'followers':followers,
                       'following':following,'totalposts':total_posts,'biodescription':bio_description,
                       'biolinks':bio_links,'bioids':bio_ids, 'biotags':bio_tags, 'bioemails':bio_emails,
                       'profilepicture':profile_picture,'privacytype':privacy_type,
                      'influencer':influencer,'buisness':buisness}) 
    #Storing results to {}.csv
    df.to_csv('result1.csv', index=False, encoding='utf-8')

In [13]:
def getAccountDataAsync(path) -> User:
    driver.get(path)
    
    user = getAccountDetails(driver, path)
    
    return user

In [28]:
def getAccountDetails(driver, url) -> User:
    content = driver.page_source
    soup = BeautifulSoup(content)

    #extract userID
    user_id = soup.find('div', {'class':'nZSzR'}).find('h2').get_text()
    #extract user name
    name = soup.find('div', {'class':'-vDIg'}).find('h1').get_text()
    
    #extract counts of posts, followers, followings
    total_posts, followers, followings, profile_pic_src = getHeaderDetails(soup)
    
    #extract bio
    bio = extractBio(soup.find('div', {'class':'-vDIg'}))
    
    bio_emails = extractEmails(bio.bio_description)
    
    influencer = True
    buisness = True
    #checking user validity
    if len(bio_emails) == 0:
        buisness = False
    if criteria(followers, followings) is False:
        influencer = False            
        
    if influencer is False and buisness is False:
        return None
    
    #if private then privacy=True
    private = checkPrivacy(soup)
    
    return User(user_id, name, followers, followings, total_posts, bio, bio_emails, profile_pic_src, private, influencer, buisness)

In [7]:
def getHeaderDetails(soup):
    page_details = soup.find('section', {'class':'zwlfE'}).find_all('li')
    
    total_posts = convert(page_details[0].find('span').get_text())
    followers = convert(page_details[1].find('span')['title'])
    followings = convert(page_details[2].find('span').get_text())
    
    profile_pic_src = soup.find('div', {'class':'XjzKX'}).find('img')['src']
    
    return total_posts, followers, followings, profile_pic_src

In [6]:
def checkPrivacy(soup):
    private = True
    if soup.find('div',{'class':'Nd_Rl _2z6nI'}) is None:
        private = False
        
    return private

In [11]:
def extractBio(soup):
    bio_links = []
    bio_ids = []
    bio_tags = []
    links = soup.find_all('a')
    for link in links:
        if "https" in link['href']:
            bio_links.append(link['href'])
            
    
    bio_description = None
    description = soup.find('span')
    
    if description != None:
        bio_description = description.get_text()  
        ids = description.find_all('a')
        if ids != None:
            for handle in ids:
                if '#' in handle.get_text():
                    bio_tags.append(handle.get_text().split('#')[1])
                elif '@' in handle.get_text():
                    bio_ids.append(handle.get_text().split('@')[1])
    
    if len(bio_links) == 0:
        bio_links = None
    if len(bio_ids) == 0:
        bio_ids = None
    if len(bio_tags) == 0:
        bio_tags = None
        
    return Bio(bio_description, bio_ids, bio_links, bio_tags)

In [21]:
def extractEmails(description):
    if description is None:
        return []
    match = re.findall(r'[\w\.-]+@[\w\.-]+', description)
    return match

In [3]:
def convert(number):
    if 'k' in number:
        return int(float(number.split('k')[0])*1000)
    if 'm' in number:
        return int(float(number.split('m')[0])*1000000)
    return int(number.replace(",",""))

In [2]:
def criteria(followers, followings):
    if followers <= 2000:
        return False
    
    if float(followings)/followers > 0.1:
        return False
    
    return True

In [25]:
#add user to list
def addUser(user):
    user_id.append(user.user_id)
    name.append(user.name)
    followers.append(user.followers)
    following.append(user.following)
    total_posts.append(user.total_posts)
    profile_picture.append(user.profile_picture)
    privacy_type.append(user.privacy_type)
    bio_description.append(user.bio.bio_description)
    bio_links.append(user.bio.bio_links)
    bio_ids.append(user.bio.bio_ids)
    bio_tags.append(user.bio.bio_tags)
    influencer.append(user.influencer)
    buisness.append(user.buisness)
    bio_emails.append(user.bio_emails)